In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import cv2
import numpy as np
import os
import re
import shutil

from models import BachNet
from models import ChopinNet

import networkx as nx
import sys
import time

from heapq import heappop as pop
from heapq import heappush as push
from utils import graph_utils
from utils import display_utils
from utils import prediction_utils
from utils import preprocessing_utils

Using TensorFlow backend.


In [4]:
train_path = "data/train"
test_path = "data/test"

input_path = "input"
output_path = "output"

gt_tag = "gt"

receptive_field_shape = (12, 12)
n_epochs = 128
save_rate = 10

In [6]:
bach = BachNet.BachNet()
bach.build(receptive_field_shape, 1)
bach.load_model('models/saved_models/Bach/model.h5')

Could not find model. Creating a new one.


IOError: Unable to create file (Unable to open file: name = 'models/saved_models/bach/model.h5', errno = 2, error message = 'no such file or directory', flags = 13, o_flags = 242)

In [ ]:
batch = dict()
input_gen = prediction_utils.input_generator(bach, train_path, input_path, gt_tag)

while True:
    try:
        f_name, img, bps, I_a, gt, gt_cuts, seeds = next(input_gen)
        graph = graph_utils.prims_initialize(img)
        batch[f_name] = img, bps, I_a, gt, gt_cuts, seeds, graph
    except StopIteration:
        break

Loading image:  slice_59
Loading image:  slice_59


In [7]:
chopin = ChopinNet.Chopin()
chopin.build(receptive_field_shape, learning_rate=1e-5)
#chopin.load_model("models/saved_model/Chopin/checkpoint")
chopin.initialize_session()

In [9]:
global_loss_timeline = []
loss_timelines = dict()
loss_file = open("data/train/chopin/global_loss.txt", 'w')
loss_file.write("f_name\tepoch\tloss\n")

for epoch in range(n_epochs):
    for f_name, (img, bps, I_a, gt, gt_cuts, seeds, graph) in batch.iteritems():
        print("Training on:", f_name)
        
        foldername = os.path.join(train_path, "chopin", f_name)
        start = time.time()
        
        loss, segmentations, cuts = chopin.train_on_image(img, I_a, gt_cuts, seeds, graph)
        
        if epoch % save_rate:
            print("Saving Model")
            chopin.save_model(os.path.join(foldername, "saved_models", "model"), epoch)
            chopin.save_model("models/saved_models/Chopin/chopin", epoch)
        
        print(time.time() - start)
        print("Loss: ", loss)
        
        plt.imsave(os.path.join(foldername, "epoch_{}_bw.png".format(epoch)), display_utils.view_boundaries(np.zeros_like(img), cuts))

        mask = display_utils.transparent_mask(display_utils.view_boundaries(img, gt_cuts), segmentations, alpha=0.75)
        plt.imsave(os.path.join(foldername, "epoch_{}_overlay.png".format(epoch)), mask)
        
        loss_file.write(f_name + "\t" + str(epoch) + "\t" + str(loss) + "\n")
        loss_file.flush()
        
        global_loss_timeline.append(loss)
        
        try:
            loss_timelines[f_name].append(loss)
        except KeyError:
            loss_timelines[f_name] = [loss]
            
        plt.plot(loss_timelines[f_name])
        plt.savefig(os.path.join(foldername, "local_loss"))
        
        plt.gcf().clear()
            
        plt.plot(global_loss_timeline)
        plt.savefig("data/train/chopin/global_loss")
        
        plt.gcf().clear()
            
loss_file.close()

Training on: 740_1460
Starting gradient segmentation...
Segmentation done: 5.726454s
Starting gradient segmentation...
Segmentation done: 0.035326s
6.30839395523
Loss:  46.7537689209
Training on: 740_1450
Starting gradient segmentation...
Segmentation done: 5.720744s
Starting gradient segmentation...
Segmentation done: 0.033008s
6.14571022987
Loss:  5.25823879242
Training on: 740_1460
Starting gradient segmentation...
Segmentation done: 5.856825s
Starting gradient segmentation...
Segmentation done: 0.035038s
Saving Model
12.9248030186
Loss:  6.20071411133
Training on: 740_1450
Starting gradient segmentation...
Segmentation done: 5.828446s
Starting gradient segmentation...
Segmentation done: 0.034598s
Saving Model
12.3658339977
Loss:  25.2894668579
Training on: 740_1460
Starting gradient segmentation...
Segmentation done: 5.767020s
Starting gradient segmentation...
Segmentation done: 0.036274s
Saving Model
13.3352379799
Loss:  4.51704454422
Training on: 740_1450
Starting gradient segmen

Segmentation done: 5.750629s
Starting gradient segmentation...
Segmentation done: 0.035316s
Saving Model
22.8120191097
Loss:  0.417468070984
Training on: 740_1450
Starting gradient segmentation...
Segmentation done: 5.768264s
Starting gradient segmentation...
Segmentation done: 0.036374s
Saving Model
25.5945081711
Loss:  0.650390565395
Training on: 740_1460
Starting gradient segmentation...
Segmentation done: 5.751294s
Starting gradient segmentation...
Segmentation done: 0.035480s
Saving Model
24.0462620258
Loss:  0.476050078869
Training on: 740_1450
Starting gradient segmentation...
Segmentation done: 5.806616s
Starting gradient segmentation...
Segmentation done: 0.035004s
Saving Model
24.4806349277
Loss:  0.558739542961
Training on: 740_1460
Starting gradient segmentation...
Segmentation done: 5.758995s
Starting gradient segmentation...
Segmentation done: 0.035356s
Saving Model
24.6667439938
Loss:  0.475578308105
Training on: 740_1450
Starting gradient segmentation...
Segmentation do

KeyboardInterrupt: 